## Consume topic from kafka -> create alerts

Alerts engine, with rules funcions list.
From kafka topic, receive list of JSONs (flights' locations), generate a dataframe from the list.
Applies each function in the rules engine list on each row in the dataframe

In [1]:
import logging
from sys import stdout
from opensky_alerts_engine import * 

formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler = logging.StreamHandler(stdout)
console_handler.setFormatter(formatter)

logger = logging.getLogger('opensky.basic_alerts')
logger.addHandler(console_handler)
logger.setLevel('DEBUG')

In [2]:
from kafka import KafkaConsumer


# # Topics/Brokers
topic_real_time_states = "real-time-states"
topic_sparse_states = 'sparse_states'
topic_brokers = ['localhost:9092']

consumer = KafkaConsumer(topic_real_time_states)



In [18]:

def abnormal_squawk(row) -> OpenSkyAlert:
    squawk_alerts_dict = {'7500' : {'subject' : 'Aircraft hijacking', 'severity' : Severity.CRITICAL}, 
                          '7600' : {'subject' : 'Radio failure', 'severity' : Severity.MEDIUM},
                          '7700' : {'subject' : 'General failure', 'severity' : Severity.HIGH},
                          None : {'subject' : 'Squawk missing', 'severity' : Severity.LOW}}
    
    try:
        squawk_alert = squawk_alerts_dict[row['squawk']]
        alert_message = f"'{squawk_alert['subject']}' report for icao24 (Mode S) : {row['icao24']}, callsign : {row['callsign']}, on_ground : {row['on_ground']}"
        return OpenSkyAlert(severity=squawk_alert['severity'], message=alert_message, alert_time=datetime.fromtimestamp(row['time']), source='abnormal_squawk')
#         if squawk_alert['severity'] != 'Low':
#             print(f"icao24 (Mode S) : {row['icao24']}, callsign : {row['callsign']}, on_ground : {row['on_ground']}, {squawk_alert}")
    except KeyError as ex:
        # Not abnormal squawk do nothing
        return None


In [19]:
def abnormal_landing(row) -> OpenSkyAlert:
    #TBD
    return None



In [22]:
import json
import pandas as pd

alerts_engine = OpenSkyAlertsEngine(severity_threshold = Severity.MEDIUM)


alerts_engine.add_rule(abnormal_squawk)
alerts_engine.add_alerts_sender(ConsoleAlertsSender())
alerts_engine.add_alerts_sender(OpenSkySqlAlertsSender(host='localhost', user='naya', passwd='naya', database='OpenSkyAlertsDB', table='alerts'))

for message in consumer:
    logger.debug('Basic Alerts: Message')
    df = pd.read_json(message.value, dtype = False)
    df.apply(alerts_engine.handle_row, axis=1)

2020-01-12 21:35:07,413 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-12 21:35:07.547752 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : ac2599, callsign : RDY62, on_ground : 0
2020-01-12 21:35:07,732 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-12 21:35:07.878923 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : ac2599, callsign : RDY62, on_ground : 0
2020-01-12 21:35:08,070 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-12 21:35:08.197945 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : ac2599, callsign : RDY62, on_ground : 0
2020-01-12 21:35:08.256783 : Alert of severity Medium occured : 'Radio failure' report for icao24 (Mode S) : a46e35, callsign : DAL2600, on_ground : 0
2020-01-12 21:35:08,378 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-12 21:35:08.514480 : Alert of severity High occured : 'General failure' report

2020-01-12 21:43:41,749 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-12 21:43:41.909677 : Alert of severity Medium occured : 'Radio failure' report for icao24 (Mode S) : a6cbdd, callsign : EJA537, on_ground : 0
2020-01-12 21:43:41.943562 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : 477ff1, callsign : BARTK03, on_ground : 0
2020-01-12 21:44:07,178 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-12 21:44:07.389233 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : 477ff1, callsign : BARTK03, on_ground : 0
2020-01-12 21:44:41,199 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-12 21:44:41.383107 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : 477ff1, callsign : BARTK03, on_ground : 0
2020-01-12 21:45:20,391 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-12 21:45:20.644544 : Alert of severity High occured : 'General failure' r

2020-01-12 21:52:22.562797 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : 3c6746, callsign : DLH7YA, on_ground : 0
2020-01-12 21:52:22.564259 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : 477ff1, callsign : BARTK03, on_ground : 0
2020-01-12 21:52:50,028 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-12 21:52:50.168158 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : a88e46, callsign : RPA3465, on_ground : 0
2020-01-12 21:52:50.265562 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : 3c6746, callsign : DLH7YA, on_ground : 0
2020-01-12 21:52:50.266709 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : 477ff1, callsign : BARTK03, on_ground : 0
2020-01-12 21:52:50.333880 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : aa8701, callsign : EJM78, on_ground : 0
2020-01-12 21:53:10

2020-01-12 21:58:25,851 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-12 21:58:26.026790 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : 477ff1, callsign : BARTK03, on_ground : 0
2020-01-12 21:58:26.095123 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : aa8701, callsign : EJM78, on_ground : 0
2020-01-12 21:58:55,898 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-12 21:58:56.110048 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : 477ff1, callsign : BARTK03, on_ground : 0
2020-01-12 21:58:56.132139 : Alert of severity Medium occured : 'Radio failure' report for icao24 (Mode S) : ab7627, callsign : AAL1274, on_ground : 1
2020-01-12 21:59:38,041 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-12 21:59:38.233995 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : 477ff1, callsign : BARTK03, on_ground : 0
2020-01-12

2020-01-12 22:08:16.551130 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : abff50, callsign : RPA5942, on_ground : 0
2020-01-12 22:08:45,822 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-12 22:08:45.945509 : Alert of severity Medium occured : 'Radio failure' report for icao24 (Mode S) : a2a30d, callsign : N2692C, on_ground : 0
2020-01-12 22:08:45.985604 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : 477ff1, callsign : BARTK03, on_ground : 0
2020-01-12 22:08:46.004502 : Alert of severity Medium occured : 'Radio failure' report for icao24 (Mode S) : a042db, callsign : DAL1786, on_ground : 0
2020-01-12 22:08:46.025217 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : abff50, callsign : RPA5942, on_ground : 0
2020-01-12 22:09:11,758 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-12 22:09:11.890867 : Alert of severity Medium occured : 'Radio failure' report f

...

2020-01-14 06:46:33.236673 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : ad7e6f, callsign : JBU2201, on_ground : 0
2020-01-14 06:46:44,819 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 06:46:56,956 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 06:47:08,872 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 06:47:20,767 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 06:47:32,748 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 06:47:45,144 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 06:47:57,357 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 06:48:09,712 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 06:48:22,247 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 06:48:22.355586 : Alert of severity Medium occured : 'Radio failure' report for icao24 (Mode S) : ab77ee, callsign : ASA0520, o

2020-01-14 07:06:10.378655 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : a335ad, callsign : ASA21, on_ground : 0
2020-01-14 07:06:30,845 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:06:30.961019 : Alert of severity High occured : 'General failure' report for icao24 (Mode S) : a335ad, callsign : ASA21, on_ground : 0
2020-01-14 07:06:50,978 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:07:11,592 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:07:32,134 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:07:52,704 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:08:13,600 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:08:34,901 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:08:55,654 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:09:16,928 - opensky.basic_alerts - DEBUG - Basic Alerts:

2020-01-14 07:27:23,661 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:27:23.753794 : Alert of severity Medium occured : 'Radio failure' report for icao24 (Mode S) : acee21, callsign : ASH5821, on_ground : 0
2020-01-14 07:27:41,572 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:27:41.661374 : Alert of severity Medium occured : 'Radio failure' report for icao24 (Mode S) : acee21, callsign : ASH5821, on_ground : 0
2020-01-14 07:27:59,734 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:27:59.823418 : Alert of severity Medium occured : 'Radio failure' report for icao24 (Mode S) : acee21, callsign : ASH5821, on_ground : 0
2020-01-14 07:28:18,028 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:28:18.113229 : Alert of severity Medium occured : 'Radio failure' report for icao24 (Mode S) : acee21, callsign : ASH5821, on_ground : 0
2020-01-14 07:28:36,316 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2

2020-01-14 07:38:02.438812 : Alert of severity Medium occured : 'Radio failure' report for icao24 (Mode S) : acee21, callsign : ASH5821, on_ground : 0
2020-01-14 07:38:24,024 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:38:24.116175 : Alert of severity Medium occured : 'Radio failure' report for icao24 (Mode S) : acee21, callsign : ASH5821, on_ground : 0
2020-01-14 07:38:45,727 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:39:07,462 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:39:29,477 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:39:51,467 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:40:13,652 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:40:36,496 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:40:48,230 - opensky.basic_alerts - DEBUG - Basic Alerts: Message
2020-01-14 07:41:00,552 - opensky.basic_alerts - DEBUG - Basic Ale

KeyboardInterrupt: 

## Backup for alerts engine base implementations

In [7]:
import logging
from sys import stdout

formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler = logging.StreamHandler(stdout)
console_handler.setFormatter(formatter)

internal_logger = logging.getLogger('opensky.alerts_engine')
internal_logger.addHandler(console_handler)
internal_logger.setLevel('INFO')

In [8]:
from enum import IntEnum
from datetime import datetime
import types
import mysql.connector


class Severity(IntEnum):
    LOW = 1
    MEDIUM = 2
    HIGH = 3
    CRITICAL = 4
    
    def __str__(self):
        return str.capitalize(self.name)
    
    def __repr__(self):
        return self.__str__()
    
    
class OpenSkyAlert:
    def __init__(self, severity: Severity, message: str, alert_time: datetime = None, source: str = 'NotDefined'):
        self._severity = severity
        self._message = message
        self._timestamp = alert_time if alert_time is not None else datetime.now()
        self._source = source
        
    @property
    def severity(self):
        return self._severity
    
    @property
    def message(self):
        return self._message
    
    @property
    def alert_time(self):
        return self._timestamp
    
    @property
    def source(self):
        return self._source

class BaseAlertsSender:
    def __init__(self, name):
        self._name = type(self)
        
    @property
    def name(self):
        return self._name

    def send(self, alert: OpenSkyAlert):
        raise NotImplemented

    @staticmethod
    def alert_message_builder(severity: Severity, message: str, alert_time: datetime = None) -> str:
        time = alert_time if alert_time is not None else datetime.now()
        return f'{time} : Alert of severity {str(severity)} occured : {message}'

    
class ConsoleAlertsSender(BaseAlertsSender):
    def __init__(self):
        super().__init__(self)
        
    @property
    def name(self):
        return self._name
    
    def send(self, alert: OpenSkyAlert):
        print(super().alert_message_builder(alert.severity, alert.message))
        return True

    
class OpenSkyAlertsEngine:
    def __init__(self, severity_threshold = Severity.MEDIUM):
        self.min_severity = severity_threshold
        self._active_rules = []
        self._active_senders = []
        
    @property
    def active_rules(self):
        return self._active_rules
    
    @property
    def active_senders(self):
        return self._active_senders
    
    def add_alerts_sender(self, alerts_sender:BaseAlertsSender):
        self._active_senders.append(alerts_sender)
         
    def add_rule(self, rule:types.FunctionType):
        """
        Add a function of form func(row: DataFrame.Row) -> OpenSkyAlert)
        """
        self._active_rules.append(rule)
    
    def handle_row(self, row):
        for rule in self.active_rules:
            try:
                alert = rule(row)
                if alert != None:
                    if alert.severity >= self.min_severity:
                        for sender in self.active_senders:
                            sender.send(alert)
            except Exception as ex:
                internal_logger.error(f'at handle_row: {ex}')
                

class OpenSkySqlAlertsSender(BaseAlertsSender):
    def __init__(self, host, user, passwd, database, table):
        super().__init__(self)
        self.host = host
        self.user = user
        self.passwd = passwd
        self.database = database
        self.table = table
        self._connection = mysql.connector.connect(host=host, user=user, passwd=passwd,
                                                   database=database, autocommit=True)
        crsr = self._connection.cursor()
        create_alerts_table = (f"""CREATE TABLE IF NOT EXISTS {table} (timestamp TIMESTAMP, source_rule NVARCHAR(255), 
                                severity NVARCHAR(16), message longtext)""")
        crsr.execute(create_alerts_table)
        internal_logger.debug(f'Alerts sender initialized, with table : {table}')
        crsr.close()

    @property
    def mysql_connection(self):
        return self._connection
        
    def send(self, alert: OpenSkyAlert):
        crsr = self._connection.cursor()
        insert_alert = f"""INSERT INTO {self.table} 
                            (timestamp, source_rule, severity, message) 
                            VALUES 
                            ('{alert.alert_time}', '{alert.source}', '{str(alert.severity)}', "{alert.message}")
                            ;
                        """
        try:
            crsr.execute(insert_alert)
        except Exception as ex:
            internal_logger.error(f'error inserting alert into mysql : {ex}')
        crsr.close()

In [80]:
import mysql.connector

sql_sender = MySqlAlertsSender(host='localhost', user='naya', passwd='naya', database='OpenSkyAlertsDB', table='alerts01')
alert = OpenSkyAlert(Severity.HIGH, message="Hello Wolrd!!!", alert_time=datetime(2020,1,12,18,0,0), source='test')


sql_sender.send(alert)

crsr.execute(f"""select * from alerts01;""")
print(crsr.fetchall())

2020-01-12 18:13:30,756 - opensky.basic_alerts - DEBUG - insert_alert query:
INSERT INTO alerts01 
                            (timestamp, source_rule, severity, message) 
                            VALUES 
                            ('2020-01-12 18:00:00', 'test', 'High', 'Hello Wolrd!!!')
                            ;
                        


DEBUG:opensky.basic_alerts:insert_alert query:
INSERT INTO alerts01 
                            (timestamp, source_rule, severity, message) 
                            VALUES 
                            ('2020-01-12 18:00:00', 'test', 'High', 'Hello Wolrd!!!')
                            ;
                        


[(datetime.datetime(2020, 1, 12, 18, 0), 'test', 'High', 'Hello Wolrd!!!'), (datetime.datetime(2020, 1, 12, 18, 0), 'test', 'High', 'Hello Wolrd!!!'), (datetime.datetime(2020, 1, 12, 18, 0), 'test', 'High', 'Hello Wolrd!!!'), (datetime.datetime(2020, 1, 12, 18, 0), 'test', 'High', 'Hello Wolrd!!!'), (datetime.datetime(2020, 1, 12, 18, 0), 'test', 'High', 'Hello Wolrd!!!'), (datetime.datetime(2020, 1, 12, 18, 0), 'test', 'High', 'Hello Wolrd!!!'), (datetime.datetime(2020, 1, 12, 18, 0), 'test', 'High', 'Hello Wolrd!!!'), (datetime.datetime(2020, 1, 12, 18, 0), 'test', 'High', 'Hello Wolrd!!!'), (datetime.datetime(2020, 1, 12, 18, 0), 'test', 'High', 'Hello Wolrd!!!'), (datetime.datetime(2020, 1, 12, 18, 0), 'test', 'High', 'Hello Wolrd!!!')]
